# ML Pipeline

In [1]:
#import libraries
import datetime

import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('omw-1.4')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import xgboost as xgb
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# import libraries
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


__Read disaster.csv__

In [2]:
df2 = pd.read_csv("disaster.csv")
df2.head()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Loading data from disaster.csv
df2.drop(['Unnamed: 0'], axis=1, inplace = True)
X = df2.message
y = df2[df2.columns[4:]]
category_names = y.columns
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=42,shuffle = True)

In [4]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. We perform `multi-label text classification` using this pipeline.

__Model Dictionary__

In [5]:
model_dict = {
    # MultiOutputClassifier
    "rfc" : MultiOutputClassifier(RandomForestClassifier()), #most robust model
    "mlp" : MultiOutputClassifier(MLPClassifier()),
    "knn" : MultiOutputClassifier(KNeighborsClassifier()),
    "xgb" : MultiOutputClassifier(xgb.XGBClassifier(objective='binary:logistic')), 
    # BinaryRelevance
    "br_mnb" : BinaryRelevance(MultinomialNB()),
}

In [6]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def hamming_loss(y_true, y_pred):
    temp=0
    for i in range(y_true.shape[0]):
        temp += np.size(y_true[i] == y_pred[i]) - np.count_nonzero(y_true[i] == y_pred[i])
    return temp/(y_true.shape[0] * y_true.shape[1])
    
def print_score(y_true, y_pred):
    print("Hamming loss: {}".format(hamming_loss(y_true, y_pred)))
    print("Hamming score: {}".format(hamming_score(y_true, y_pred)))
    print("---") 
     

In [7]:
def train_multioutput(model_str):

    model = model_dict[model_str]

    pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',model)
    ])
    
    pipeline.fit(X_train, y_train)
    pred_val = pipeline.predict(X_val)
    true_val = y_val.values
    
    print(f"For {model_str}:")
    
    if print_score == 'br_mnb':
        print_score(true_val, pred_val.toarray())
    else:
        print_score(true_val, pred_val)
        
    
    return pipeline

__Prediction using Disaster Response Pipeline__

In [8]:
pipeline = train_multioutput(model_str = "xgb")

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:05:05] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:05:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:05:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:05:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old b

In [ ]:
pipeline = train_multioutput(model_str = "knn")

For KNN:
Hamming loss: 0.06658932960420375
Hamming score: 0.43871978265442335
---


In [ ]:
pipeline = train_multioutput(model_str = "rfc")

For: rfc
Hamming loss: 0.0543107466734469
Hamming score: 0.4904770451899944
---


In [13]:
pipeline = train_multioutput(model_str = "br_mnb")

For br_mnb:
Hamming loss: 0.06485189422832444
Hamming score: 0.3852646642149772
---


#### Predict Label of given message

In [12]:
def predict_query(pipeline):
    
    if pipeline is None:
        print("No pipeline found")
        return ;
        
    query = input('Please enter your message:\n')

    y_pred = pipeline.predict([query])[0]

    print("The sentence belongs to following labels:\n")
    for i in range(y_pred.shape[0]):
        if(y_pred[i] == 1):
            print(y.columns[i])

_Example 1_

query1 = 11 people died in storm. We are hungry and need food

In [13]:
predict_query(pipeline)

Please enter your message:
 11 people died in storm. We are hungry and need food


The sentence belongs to following labels:

('related',)
('request',)
('aid_related',)
('food',)
('death',)
('storm',)
('direct_report',)


_Example 2_

query2 = We're asking for water, medical supply, food

In [14]:
predict_query(pipeline)

Please enter your message:
 We're asking for water, medical supply, food


The sentence belongs to following labels:

('related',)
('request',)
('aid_related',)
('medical_products',)
('water',)
('food',)
('direct_report',)


_Example 3_

query = Emergency, aux cayes is flooding, they needs help.

In [15]:
predict_query(pipeline)

Please enter your message:
 Emergency, aux cayes is flooding, they needs help.


The sentence belongs to following labels:

('related',)
('request',)
('aid_related',)
('other_aid',)
('weather_related',)
('floods',)


__Scope of future research and development__

### Hyperparameter Tuning using GridSearchCV

In [7]:
# params_grid = {

#     #only for multi-output classifiers

#     'rfc' : {
#         'clf__estimator__n_estimators': [50, 100, 150],
#         'clf__estimator__min_samples_split': [2, 3, 4],
#     },

#     'knn' : {
#         'clf__estimator__n_neighbors': [3, 5, 7, 9, 11] # usually odd numbers
#     },
    
#     'xgb' : {
#            'clf__estimator__max_depth':[5,10],
#            'clf__estimator__n_estimators':[100],
#            'clf__estimator__learning_rate':[0.05,0.1],
#            'clf__estimator__colsample_bytree':[0.8,0.95]
#             }
# }

In [9]:
# def grid_search_cv(model_str):
    
#     model = model_dict[model_str]

#     pipeline = Pipeline([
#         ('vect',CountVectorizer(tokenizer = tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf',model)
#     ])
    
#     gs = GridSearchCV(pipeline, param_grid=params_grid[model_str])
    
#     gs.fit(X_train, y_train)
#     print(gs.grid_scores_)
    
#     #finding the best paramesters based on grip search
#     print(gs.best_params_)
    
#     #building new model
#     optimised_model = gs.best_estimator_
#     print(optimised_model)
    
#     #pickle optimised model
#     pickle.dump(optimised_model, open("ht_" + model_str, 'wb'))
    
#     #evaluate model on val/test set
#     pred_val = optimised_model.predict(X_val)
#     true_val = y_val.values
    
#     print(f"Grid Search for: {model_str}")
#     print_score(true_val, pred_val)
    

In [ ]:
# grid_search_cv(model_str = "rfc")

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 

...

building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

In [ ]:
# grid_search_cv(model_str = "knn")

In [ ]:
# grid_search_cv(model_str = "xgb")

#### Prediction on query label

In [ ]:
# optimised_pred = optimised_model.predict([query])[0]
# print("The sentence belongs to following labels:")
# for i in range(optimised_pred.shape[0]):
#     if(optimised_pred[i] == 1):
#         print(y.columns[i])